# Кластеризация ЕГЭ (8.5 если сделаны все задачи)

Рядом лежат экселевские таблицы с координатами точек. В 3 и 4 задаче используется манхетеское расстояние. В остальных Евклидово.

* Постройте Распределение точек
* Кластеризуйте руками
* Постройте Распределение точек, отметьте принадлежность кластеров цветами
* Найдите среди в каждом кластере точку расстояние от которой до всех остальных минимально
* Выведите усреднение центров всех кластеров * 1000

In [3]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

file_path = '/EGE/0.xls'
df = pd.read_excel(file_path, decimal=',')

x = df.iloc[:, 0].dropna().to_numpy()
y = df.iloc[:, 1].dropna().to_numpy()
#print("Массив X:", x)
#print("Массив Y:", y)

def center(coords):
  sumx = 0
  sumy = 0
  for i in range(len(coords)):
    sumx += coords[i][0]
    sumy += coords[i][1]
  return (int(sumx / len(coords) * 1000), int(sumy / len(coords) * 1000))

def under_line(x, y):
    return (8 * x - 3 * y + 11 > 0)

red = []
blue = []
for i in range(len(x)):
  if under_line(x[i], y[i]):
    red.append([x[i], y[i]])
  else:
    blue.append([x[i], y[i]])

best_red = (-1, -1)
best_sum_red = 1e18
for cur in range (len(red)):
  sum = 0
  for i in range(len(red)):
    sum += math.dist(red[cur], red[i])
  if sum < best_sum_red:
    best_sum_red = sum
    best_red = red[cur]

best_blue = (-1, -1)
best_sum_blue = 1e18
for cur in range (len(blue)):
  sum = 0
  for i in range(len(blue)):
    sum += math.dist(blue[cur], blue[i])
  if sum < best_sum_blue:
    best_sum_blue = sum
    best_blue = blue[cur]

print(center([best_red, best_blue]))

plt.scatter([t[0] for t in blue], [t[1] for t in blue], color='blue', label='First')
plt.scatter([t[0] for t in red], [t[1] for t in red], color='red', label='Second')
plt.scatter(best_red[0], best_red[1], color='green', marker='x', s=100, label=f'Center_red')
plt.scatter(best_blue[0], best_blue[1], color='purple', marker='x', s=100, label=f'Center_blue')

plt.legend()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/EGE/0.xls'

# Кластеризация (0.75 за каждый алгоритм на всех данных)

На предложенных распределениях данных проверьте предложенные алгоритмы. Постройте графики кластеризации для каждой пары алгоритм-данные, разные кластеры покрасьте разным цветом. Воспользуйтесь sklearn реализациями. Параметры кластеризации для разных алгоритмов подберите такие, чтобы алгоритмы можно было сравнивать (по возможности одинаковое количество кластеров и т.д.)

In [ ]:
import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

In [ ]:
two_means =  
dbscan = 

clustering_algorithms = (
        ("MiniBatch\nKMeans", two_means),
        ("DBSCAN", dbscan),
    )

In [ ]:
np.random.seed(0)

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=0.5, noise=0.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=0.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 13))
plt.subplots_adjust(
    left=0.02, right=0.98, bottom=0.001, top=0.95, wspace=0.05, hspace=0.01
)

plot_num = 1

default_base = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 10,
    "n_clusters": 3,
    "min_samples": 20,
    "xi": 0.05,
    "min_cluster_size": 0.1,
}

datasets = [
    (
        noisy_circles,
        {
            "damping": 0.77,
            "preference": -240,
            "quantile": 0.2,
            "n_clusters": 2,
            "min_samples": 20,
            "xi": 0.25,
        },
    ),
    (noisy_moons, {"damping": 0.75, "preference": -220, "n_clusters": 2}),
    (
        varied,
        {
            "eps": 0.18,
            "n_neighbors": 2,
            "min_samples": 5,
            "xi": 0.035,
            "min_cluster_size": 0.2,
        },
    ),
    (
        aniso,
        {
            "eps": 0.15,
            "n_neighbors": 2,
            "min_samples": 20,
            "xi": 0.1,
            "min_cluster_size": 0.2,
        },
    ),
    (blobs, {}),
    (no_structure, {}),
]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params["quantile"])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params["n_neighbors"], include_self=False
    )
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    for name, algorithm in clustering_algorithms:
        t0 = time.time()